In [ ]:
# Package Author Mark Benmuvhar
# thesis_filter_pats
# Version 2.0.0
# 10/16/2022

In [1]:
import numpy as np
import pandas as pd
from datetime import date
import re
import os

In [2]:
# pdf_io
directory = ('OrangeData\Pats')
start=True
for filename in os.scandir(directory):
    print(filename)
    if not filename.is_dir():
        data_in=pd.read_csv(
            filename,
            dtype =
               {
                'DP_Name' : str,
                'Application_No' : str,
                'Product_No': int,
                'Patent_No': str,
                'Patent_Expiration_Date': str, 
                'Patent_Codes': str, 
                'Patent_Delist' : str,
                'Exclusivity' : str,
                'Exclusive_Expire': str
              },
            parse_dates = True
        )
        if start == True:
            df = data_in.copy()
            start = False
        else:
            df = pd.concat([df, data_in])


        
                           

<DirEntry 'OB-Annual-2005-25th-Ed_pat_1_5_1.csv'>
<DirEntry 'OB-Annual-2006-26th-Ed_pat_1_5_1.csv'>
<DirEntry 'OB-Annual-2007-27th-Ed_pat_1_5_1.csv'>
<DirEntry 'OB-Annual-2008-28th-Ed_pat_1_5_1.csv'>
<DirEntry 'OB-Annual-2009-29th-Ed_pat_1_5_1.csv'>
<DirEntry 'OB-Annual-2010-30th-Ed_pat_1_5_1.csv'>
<DirEntry 'OB-Annual-2011-31st-Ed_pat_1_5_1.csv'>
<DirEntry 'OB-Annual-2012-32nd-Ed_pat_1_5_1.csv'>
<DirEntry 'OB-Annual-2013-33rd-Ed_pat_1_5_1.csv'>
<DirEntry 'OB-Annual-2014-34th-Ed_pat_1_5_1.csv'>
<DirEntry 'OB-Annual-2015-35th-Ed_pat_1_5_1.csv'>
<DirEntry 'OB-Annual-2016-36th-Ed_pat_1_5_1.csv'>
<DirEntry 'OB-Annual-2017-37th-Ed_pat_1_5_1.csv'>
<DirEntry 'OB-Annual-2018-38th-Ed_pat_1_5_1.csv'>
<DirEntry 'OB-Annual-2019-39th-Ed_pat_1_5_1.csv'>
<DirEntry 'OB-Annual-2020-40th-Ed_pat_1_5_1.csv'>
<DirEntry 'OB-Annual-2021-41st-Ed_pat_1_5_1.csv'>
<DirEntry 'OB-Annual-2022-42nd-Ed_pat_1_5_1.csv'>


In [3]:
# find_use_pats looks for usage code flag via regex
# Intended for use in an apply statement using the Patent_Code column containing str

def find_use_pat(s):
    p=re.compile('U-\d+')
    try:
        if pd.notna(s):
            m=p.search(s)
            if m:
                return m[0]
        return np.nan
    except:
        print('error getting use code from ', s,'\n Found ',m[0])

In [4]:
def find_app_type(s):
    try:
        if pd.notna(s):
            p=re.compile(r'A|N')
            m=p.search(s)
            if m:
                return m[0]
        return np.nan
    except:
        print('error finding application type in ', s)

In [5]:
def find_app_nums(s):
    try:
        if pd.notna(s):
            p=re.compile(r'\d+')
            m=p.search(s)
            return m[0]
        else:
            return np.nan
    except:
        print('error finding application numbers in ', s)

In [6]:
df['Patent_Delist'].replace(np.nan, False, inplace = True)
dfw = df.drop_duplicates(keep = 'first', 
                         subset  = df.columns[~df.columns.str.contains('Product_No')])
dfw.reset_index(inplace = True, drop = True)

In [11]:
#https://www.statology.org/pandas-filter-rows-containing-string/
dp = dfw['Patent_Codes'].str.contains('DP', na=False)
dp.name = 'DP'
ds = dfw['Patent_Codes'].str.contains('DS', na=False)
ds.name = 'DS'
use = dfw['Patent_Codes'].apply(find_use_pat)
use.name = 'Use'
pn = dfw['Patent_No'].apply(find_app_nums)
pn.name = 'Pat_Nums'
pe = dfw['Patent_No'].str.contains('PED')
pe.name = 'PE'

dfw=pd.concat([dfw, dp], axis = 1)
dfw=pd.concat([dfw, ds], axis = 1)
dfw=pd.concat([dfw, use], axis = 1)
dfw=pd.concat([dfw, pn], axis = 1)
dfw=pd.concat([dfw, pe], axis = 1)

In [13]:
app_type = dfw['Application_No'].apply(find_app_type)
app_nums = dfw['Application_No'].apply(find_app_nums)
dfw['app_type']= app_type
dfw['app_nums']= app_nums

In [14]:
dfw.drop(['Unnamed: 0', 'Application_No','Patent_Codes', 'Patent_No'],
         axis = 1, inplace = True)

In [15]:
dfw.to_csv('OrangeData\\for_analysis\ob_pats_condensed_2_0.csv')